# Sprint-3：線形回帰のスクラッチ
___

## 問題1：仮定関数

### import部

In [35]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

from sklearn.linear_model import LinearRegression

### クラス：ScratchLinearRegression

In [135]:
class ScratchLinearRegression():
    """
    線形回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ
    self.loss : 次の形のndarray, shape (self.iter,)
      学習用データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証用データに対する損失の記録

    """

    def __init__(self, num_iter=100000, lr=0.01, bias=False, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.bias = bias
        self.verbose = verbose
        # 損失を記録する配列を用意
        self.loss = np.zeros(self.iter)
        self.val_loss = np.zeros(self.iter)
        
        self.theta = np.array(0)

    def fit(self, X, y, X_val=None, y_val=None):
        """
        線形回帰を学習する。検証用データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            学習用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            学習用データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証用データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証用データの正解値
        """
        # Set theta with random value
#         onesX = np.array([np.concatenate(([1], v)) for v in X])
#         self.theta = np.random.random(len(onesX[0]))

        self.theta = np.random.random(len(X[0])+1)
        
        count = 0
        self.loss[count] = (0.5*MSE(self._linear_hypothesis(X), y))
        while count < self.iter:
            error = self._linear_hypothesis(X) - y
#             print(error.shape)
            self._gradient_descent(X, error)
            self.loss[count] = (0.5*MSE(self._linear_hypothesis(X), y))

            count += 1
        
#         self.loss[count] = (0.5*MSE(self._linear_hypothesis(onesX), y))
#         while count < self.iter:
#             self._gradient_descent(onesX, self.loss[count])
#             self.loss[count] = (0.5*MSE(self._linear_hypothesis(onesX), y))

#             count += 1
        
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print(self._linear_hypothesis(onesX))
            
        return self.loss

    def predict(self, X):
        """
        線形回帰を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """

        return self._linear_hypothesis(X)
    
    def _linear_hypothesis(self, X):
        """
        線形の仮定関数を計算する

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          学習データ

        Returns
        -------
          次の形のndarray, shape (n_samples, 1)
          線形の仮定関数による推定結果

        """

        onesX = np.array([np.concatenate(([1], v)) for v in X])
#         h_theta_x = np.dot(X, self.theta)
        h_theta_x = np.dot(onesX, self.theta)

        return h_theta_x
    
    def _gradient_descent(self, X, error):
        """
        説明を記述
        """

        # Update theta
        self.theta = self.theta - np.sum(np.dot(self._linear_hypothesis(X) - error, X))/len(X)*self.lr
    
def MSE(y_pred, y):
    """
    平均二乗誤差の計算

    Parameters
    ----------
    y_pred : 次の形のndarray, shape (n_samples,)
      推定した値
    y : 次の形のndarray, shape (n_samples,)
      正解値

    Returns
    ----------
    mse : numpy.float
      平均二乗誤差
    """
    
    mse = (1/len(y_pred)) * np.sum((y - y_pred)**2)
    
    return mse

### read csv

In [4]:
train_dataset = pd.read_csv("../Week3/application_train.csv")
test_dataset = pd.read_csv("../Week3/application_test.csv")

### データの前処理

In [5]:
# Fill missings
train_dataset_filled = train_dataset.fillna(-1)
test_dataset_filled = test_dataset.fillna(-1)

# Replace categorical value to dummy value
for col in train_dataset_filled.columns:
    labels, uniques = pd.factorize(train_dataset_filled[col])
    train_dataset_filled[[col]] = labels

for col in test_dataset_filled.columns:
    labels, uniques = pd.factorize(test_dataset_filled[col])
    test_dataset_filled[[col]] = labels

### 学習用にデータを整形

In [6]:
train_X = train_dataset_filled.drop(["TARGET"], axis=1)
train_Y = train_dataset_filled["TARGET"]

### 学習用とテスト用のデータに分割

In [9]:
x_train, x_test, y_train, y_test = train_test_split(np.array(train_X), \
                                                    np.array(train_Y), \
                                                    train_size=0.70, test_size=0.30)

In [144]:
slr = ScratchLinearRegression(num_iter=100, lr=0.001)
check_temp = slr.fit(x_train, y_train)

In [145]:
pred = slr.predict(x_test)

In [146]:
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label=1)

In [147]:
auc(fpr, tpr)

0.4974132339138688

In [142]:
lr_model = LinearRegression()
lr_model = lr_model.fit(x_train, y_train)
pred1 = lr_model.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test, pred1, pos_label=1)

In [143]:
auc(fpr, tpr)

0.6564202689512589